In [1]:
# Importar librerías para manipulación y análisis de datos
# -----------------------------------------------------------------------
import pandas as pd
from pandas import DataFrame
import numpy as np
pd.set_option('display.max_columns', None)  # Establece una opción de Pandas para mostrar todas las columnas de un DataFrame.

# Imputación de nulos usando métodos avanzados estadísticos
# -----------------------------------------------------------------------
from sklearn.impute import SimpleImputer
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.impute import KNNImputer

# Librerías de visualización
# -----------------------------------------------------------------------
import seaborn as sns
import matplotlib.pyplot as plt

# establece el estilo del fondo de las gráficas como "whitegrid". Las gráficas tendrán un fondo blanco con líneas de cuadrícula horizontales y verticales.
sns.set_style("whitegrid")

# cambiará el esquema de colores predeterminado de Seaborn a uno más oscuro, adecuado para gráficas con fondos claros. 
sns.set_theme(style="dark")

# establece el estilo en Solarize_Lihgt2. 
plt.style.use('Solarize_Light2')

In [2]:
df_data_original =pd.read_csv("DATA.CSV",index_col=0)
df_data_original.head(2).T

,0,1
Age,51,52
Attrition,No,No
BusinessTravel,NaN,NaN
DailyRate,"684,0$","699,0$"
Department,NaN,NaN
DistanceFromHome,6,1
Education,3,4
EducationField,NaN,Life Sciences
employeecount,1,1
employeenumber,"162,0","259,0"


In [3]:
df_data_original.columns

Index(['Age', 'Attrition', 'BusinessTravel', 'DailyRate', 'Department',
       'DistanceFromHome', 'Education', 'EducationField', 'employeecount',
       'employeenumber', 'EnvironmentSatisfaction', 'Gender', 'HourlyRate',
       'JobInvolvement', 'JobLevel', 'JobRole', 'JobSatisfaction',
       'MaritalStatus', 'MonthlyIncome', 'MonthlyRate', 'NUMCOMPANIESWORKED',
       'Over18', 'OverTime', 'PercentSalaryHike', 'PerformanceRating',
       'RelationshipSatisfaction', 'StandardHours', 'StockOptionLevel',
       'TOTALWORKINGYEARS', 'TrainingTimesLastYear', 'WORKLIFEBALANCE',
       'YearsAtCompany', 'YearsInCurrentRole', 'YearsSinceLastPromotion',
       'YEARSWITHCURRMANAGER', 'SameAsMonthlyIncome', 'DateBirth', 'Salary',
       'RoleDepartament', 'NUMBERCHILDREN', 'RemoteWork'],
      dtype='object')

# 1.Homogeneización del nombre de las columnas (en minúsculas)

In [4]:
df_data_copy= df_data_original.copy()


In [5]:
df_data_copy.head(2)

,Age,Attrition,BusinessTravel,DailyRate,Department,DistanceFromHome,Education,EducationField,employeecount,employeenumber,EnvironmentSatisfaction,Gender,HourlyRate,JobInvolvement,JobLevel,JobRole,JobSatisfaction,MaritalStatus,MonthlyIncome,MonthlyRate,NUMCOMPANIESWORKED,Over18,OverTime,PercentSalaryHike,PerformanceRating,RelationshipSatisfaction,StandardHours,StockOptionLevel,TOTALWORKINGYEARS,TrainingTimesLastYear,WORKLIFEBALANCE,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YEARSWITHCURRMANAGER,SameAsMonthlyIncome,DateBirth,Salary,RoleDepartament,NUMBERCHILDREN,RemoteWork
0,51,No,NaN,"684,0$",NaN,6,3,NaN,1,"162,0",1,0,51,3,5,resEArch DIREcToR,3,NaN,"19537,0",6462,7,Y,No,13,"3,0",3,NaN,0,NaN,5,"3,0",20,NaN,15,15,"19537,0",1972,1000000000$,NaN,NaN,Yes
1,52,No,NaN,"699,0$",NaN,1,4,Life Sciences,1,"259,0",3,0,65,2,5,ManAGeR,3,NaN,"19999,0",5678,0,NaN,NaN,14,"3,0",1,NaN,1,"34,0",5,"3,0",33,NaN,11,9,"19999,0",1971,1000000000$,NaN,NaN,1


In [6]:
nuevas_columnas={columna:columna.lower() for columna in df_data_copy} #modificamos las columnas a minúsculas
df_data_copy.rename(columns= nuevas_columnas, inplace= True) #cambiamos las columnas antiguas por las homogeneizadas en el paso anterior (nuevas_columnas)

In [7]:
df_data_copy.head(2).T

,0,1
age,51,52
attrition,No,No
businesstravel,NaN,NaN
dailyrate,"684,0$","699,0$"
department,NaN,NaN
distancefromhome,6,1
education,3,4
educationfield,NaN,Life Sciences
employeecount,1,1
employeenumber,"162,0","259,0"


# 2. Limpieza y verificación de datos
- 2.1 Modificar columnas categóricas a numéricas
- 2.2 Homogeneizar las etiquetas de las columnas categóricas
- 2.3 Distance from home - cambiar los valores negativos a positivos -- Nuestra decisión es que hubo un error de inserción pero q son valores positivos
- 2.4 Creación de nuevas columnas
- 2.5 Environmental satisfaction
- 2.6 Verificación de categorías
- 2.7 Duplicados
- 2.8 Nulos

#### 2.1 Cambiar de strings (objeto) a numéricas

In [8]:
#____STRING A INT_________
#age:               12 valores en letras: cambiarlos a nros + convertir todos los nros a int.
#worklifebalance quitar ",0" y convertir a int.

#____STRING A FLOAT_________
#totalworkingyears: cambiar comas por puntos
#daily rate (comas por puntos y quitarle el símbolo del dolar)
#hourly rate
#monthlyincome
#monthlyrate

In [9]:
#Age:
def texto_nro(cadena):
    cadena = cadena.replace("twenty-six","26").replace("twenty-four","24").replace("thirty-two","32").replace("thirty-six","36").replace("thirty-seven","37").replace("thirty-one","31").replace("thirty","30").replace("forty-seven","47").replace("fifty-two","52").replace("fifty-five","55").replace("fifty-eight","58")
    return int(cadena)

df_data_copy["age"] = df_data_copy["age"].apply(texto_nro)


In [10]:
#Worklifebalance:  ### AUNQUE LO DIGA QUE A UN INT. LO CONVIERTE A FLOAT

def nro(cadena):

    try: 
        cadena = cadena.replace(",0","")
        return int(cadena)
    
    except: 
        return np.nan #tiene varios valores que son "nan"

df_data_copy["worklifebalance"] = df_data_copy["worklifebalance"].apply(nro)

In [11]:
# totalworkingyears", "monthlyincome", "monthlyrate: Cambio de string a float

def decimal(cadena):

    try: 
        return float(cadena.replace(",",".").replace("$","."))

    except: 
        return np.nan #tiene varios valores que son "nan"
    
columnas_decimal = ["totalworkingyears", "monthlyincome"]

for columna in columnas_decimal:
    df_data_copy[columna] = df_data_copy[columna].apply(decimal)

In [12]:
# "hourlyrate", "monthlyrate" : cambio de string a float - no tienen decimales

def decimal3(cadena):
    try:
        return float(cadena)
    except:
        return np.nan

col_dec_ent = ["hourlyrate", "monthlyrate"]

for columna in col_dec_ent:
    df_data_copy[columna] = df_data_copy[columna].apply(decimal3)

In [13]:
## Nota: hourlyrate - parecía en el original como si no hubiese nulos pero había algunos que eran "Not available" y q contaban como interger

#### 2.2 Homogeneizar categóricas

In [14]:
#Columnas a homogeneizar sus categorías:
# educationfield (minúsculas)
# jobrole (minúsculas)
# maritalstatus(minúsculas y homogeneización)

def homogeneizar(cadena):
    try:
        return cadena.lower().replace("marreid","married")
    except:
        return np.nan

col_homo = ["educationfield", "jobrole", "maritalstatus"]

for columna in col_homo:
    df_data_copy[columna] = df_data_copy[columna].apply(homogeneizar)

In [15]:
# Homogeneizar "remotework" a 2 opciones: Yes, No
df_data_copy["remotework"] = df_data_copy["remotework"].replace({"True": "Yes", 'False': "No", '0': "No", '1': "Yes"})

#### 2.3 Distance from home - cambiar negativos a positivos

In [16]:
def cambiar_positivos(numero):
    try:
        return abs(numero) #"abs" es un método de pandas para cambiar de negativo a positivo
    except:
        return np.nan

df_data_copy["distancefromhome"] = df_data_copy["distancefromhome"].apply(cambiar_positivos)

In [17]:
#OPCIÓN 2_alternativa
# df_data_copy["distancefromhome"] = df_data_copy["distancefromhome"].apply(lambda x: int(str(x).replace("-", "")) if "-" in str(x) else int(x))
# df_data_copy

#### 2.4 Creación de nuevas columnas

In [18]:
#1. GÉNERO

#Creamos la función 
def categorizar_gender (numero):
    if numero == 0:
        return 'male'
    else:
        return 'female'




In [19]:
#Creación de una columna con la función
df_data_copy.insert(13, 'gender_cat', df_data_copy["gender"].apply(categorizar_gender))


#### 2.5 Environmentalsatisfaction (cambiar a nulos los que no son 1-4)

In [20]:

def convertir_nulos (numero):
    if numero == 1:
        return numero
    elif numero == 2: 
        return numero
    elif numero == 3:
        return numero
    elif numero == 4:
        return numero
    else:
        return np.nan

#Creación de una columna con la función
df_data_copy["environmentsatisfaction"] = df_data_copy["environmentsatisfaction"].apply(convertir_nulos)

In [21]:
#df_data_copy["environmentsatisfaction"].isnull().sum()

#### 2.6 Verificación de clasificaciones categóricas
- education (1 menos y 5 más educación)
    Parece que es así pero hay personas con un nivel "1" de educación que son directores y managers y tb 1 q tienenn altos "hourlyrates"

- jobinvolvement (1 menos y 4 máximo) -  a priori
    No puedo determinarlo, no parece q esté unico a jobsatisfaction

- jobsatisfaction (1 mínimo y 4 máximo) - a priori
    No puedo determinarlo, no parece q esté unico a jobsatisfaction

- joblevel (1 mínimo y 5 máximo) - a priori
    Sí, el 1 son técnicos el 5 son managers, directores

- relationshiptsatisfaction (1 mínimo y 4 máximo) - a priori
    No guarda una relación directa con otras columnas

- stockoption (0 sin opciones y 1-3 de pocas acciones a muchas) - a priori
No guarda una relación directa con otras columnas. Hay empleados q llevan 0 años trabjando y tienen el nivel 3 de opciones. y tampoco por el tipo de posición hay relación

- workinglifebalance (1 mínimo -4 máximo) -a prioru
tampoco existe relación directa con otras columnas

In [22]:
#2. environmentsatisfaction,relationshipsatisfaction,worklifebalance

def categorizar_environmentsatisfaction(valor):

    if valor == 4:
        return "Maximun"
    elif valor == 3:
        return "Medium"
    elif valor == 2:
        return "Little"
    elif valor == 1:
        return "Nothing"
    else:
        return "Unknown"

#df_data_copy["environmentsatisfaction_cat"] = df_data_copy["environmentsatisfaction"].apply(categorizar_environmentsatisfaction)
#df_data_copy["relationshipsatisfaction_cat"] = df_data_copy["relationshipsatisfaction"].apply(categorizar_environmentsatisfaction)
#df_data_copy["worklifebalance_cat"] = df_data_copy["worklifebalance"].apply(categorizar_environmentsatisfaction)

df_data_copy.head(2)

df_data_copy.insert(11, 'environmentsatisfaction_cat', df_data_copy["environmentsatisfaction"].apply(categorizar_environmentsatisfaction))
df_data_copy.insert(27, 'relationshipsatisfaction_cat', df_data_copy["relationshipsatisfaction"].apply(categorizar_environmentsatisfaction))


In [23]:
df_data_copy.insert(27, 'relationshipsatisfaction_cat', df_data_copy["relationshipsatisfaction"].apply(categorizar_environmentsatisfaction))


ValueError: cannot insert relationshipsatisfaction_cat, already exists

In [ ]:
df_data_copy.insert(33, 'worklifebalance_cat', df_data_copy["worklifebalance"].apply(categorizar_environmentsatisfaction))


In [ ]:
df_data_copy

,age,attrition,businesstravel,dailyrate,department,distancefromhome,education,educationfield,employeecount,employeenumber,environmentsatisfaction,environmentsatisfaction_cat,gender,gender_cat,hourlyrate,jobinvolvement,joblevel,jobrole,jobsatisfaction,maritalstatus,monthlyincome,monthlyrate,numcompaniesworked,over18,overtime,percentsalaryhike,performancerating,relationshipsatisfaction,relationshipsatisfaction_cat,standardhours,stockoptionlevel,totalworkingyears,trainingtimeslastyear,worklifebalance,worklifebalance_cat,yearsatcompany,yearsincurrentrole,yearssincelastpromotion,yearswithcurrmanager,sameasmonthlyincome,datebirth,salary,roledepartament,numberchildren,remotework
0,51,No,NaN,"684,0$",NaN,6,3,NaN,1,"162,0",1.0,Nothing,0,male,51.0,3,5,research director,3,NaN,19537.0,6462.0,7,Y,No,13,"3,0",3,Medium,NaN,0,NaN,5,3.0,Medium,20,NaN,15,15,"19537,0",1972,1000000000$,NaN,NaN,Yes
1,52,No,NaN,"699,0$",NaN,1,4,life sciences,1,"259,0",3.0,Medium,0,male,65.0,2,5,manager,3,NaN,19999.0,5678.0,0,NaN,NaN,14,"3,0",1,Nothing,NaN,1,34.0,5,3.0,Medium,33,NaN,11,9,"19999,0",1971,1000000000$,NaN,NaN,Yes
2,42,No,travel_rarely,"532,0$",Research & Development,4,2,technical degree,1,"319,0",3.0,Medium,0,male,58.0,3,5,manager,4,married,19232.0,4933.0,1,NaN,No,11,"3,0",4,Maximun,NaN,0,22.0,3,NaN,Unknown,22,NaN,11,15,"19232,0",1981,1000000000$,ManaGER - Research & Development,NaN,Yes
3,47,No,travel_rarely,"359,0$",NaN,2,4,medical,1,NaN,1.0,Nothing,1,female,82.0,3,4,research director,3,married,17169.0,26703.0,3,Y,NaN,19,"3,0",2,Little,NaN,2,NaN,2,NaN,Unknown,20,NaN,5,6,"17169,0",1976,1000000000$,NaN,NaN,No
4,46,No,NaN,"1319,0$",NaN,3,3,technical degree,1,NaN,1.0,Nothing,1,female,45.0,4,4,sales executive,1,divorced,NaN,7739.0,2,Y,No,12,"3,0",4,Maximun,NaN,1,NaN,5,3.0,Medium,19,NaN,2,8,NaN,1977,1000000000$,NaN,NaN,No
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1609,36,Yes,travel_rarely,"530,0$",NaN,3,1,life sciences,1,"967,0",3.0,Medium,0,male,51.0,2,3,sales executive,4,married,10325.0,5518.0,1,Y,NaN,11,NaN,1,Nothing,NaN,1,NaN,6,3.0,Medium,16,NaN,3,7,"10325,0",1987,1000000000$,NaN,NaN,No
1610,45,No,non-travel,"805,0$",NaN,4,2,NaN,1,"972,0",3.0,Medium,0,male,57.0,3,2,laboratory technician,2,NaN,4447.0,23163.0,1,NaN,NaN,12,"3,0",2,Little,NaN,0,NaN,5,2.0,Little,9,NaN,0,8,"4447,0",1978,1000000000$,NaN,NaN,Yes
1611,39,No,travel_rarely,"903,0$",NaN,13,5,NaN,1,NaN,NaN,Unknown,0,male,41.0,4,3,sales executive,3,single,NaN,2560.0,0,NaN,No,18,"3,0",4,Maximun,NaN,0,9.0,3,3.0,Medium,8,NaN,0,7,NaN,1984,1000000000$,NaN,NaN,Yes
1612,36,No,non-travel,"1229,0$",NaN,8,4,technical degree,1,"990,0",1.0,Nothing,0,male,84.0,3,2,sales executive,4,divorced,NaN,25952.0,4,NaN,No,13,NaN,4,Maximun,NaN,2,12.0,3,3.0,Medium,7,NaN,0,7,NaN,1987,1000000000$,NaN,NaN,Yes


In [ ]:
for indice, columns in enumerate (df_data_copy.columns):
    print(indice, columns)

0 age
1 attrition
2 businesstravel
3 dailyrate
4 department
5 distancefromhome
6 education
7 education_cat
8 educationfield
9 employeecount
10 employeenumber
11 environmentsatisfaction
12 environmentsatisfaction_cat
13 gender
14 gender_cat
15 hourlyrate
16 jobinvolvement
17 joblevel
18 jobrole
19 jobsatisfaction
20 maritalstatus
21 monthlyincome
22 monthlyrate
23 numcompaniesworked
24 over18
25 overtime
26 percentsalaryhike
27 performancerating
28 relationshipsatisfaction
29 relationshipsatisfaction_cat
30 standardhours
31 stockoptionlevel
32 totalworkingyears
33 trainingtimeslastyear
34 worklifebalance
35 worklifebalance_cat
36 yearsatcompany
37 yearsincurrentrole
38 yearssincelastpromotion
39 yearswithcurrmanager
40 sameasmonthlyincome
41 datebirth
42 salary
43 roledepartament
44 numberchildren
45 remotework
46 jobinvolvement_cat


In [ ]:
#education
def categorizar_education(valor):
    if valor == 5:
        return "University"
    elif valor == 3:
        return "college"
    elif valor == 2:
        return "high school"
    elif valor == 1:
        return "middle school"
    else:
        return "Unknown"



df_data_copy.head(2)

,age,attrition,businesstravel,dailyrate,department,distancefromhome,education,educationfield,employeecount,employeenumber,environmentsatisfaction,environmentsatisfaction_cat,gender,gender_cat,hourlyrate,jobinvolvement,joblevel,jobrole,jobsatisfaction,maritalstatus,monthlyincome,monthlyrate,numcompaniesworked,over18,overtime,percentsalaryhike,performancerating,relationshipsatisfaction,relationshipsatisfaction_cat,standardhours,stockoptionlevel,totalworkingyears,trainingtimeslastyear,worklifebalance,worklifebalance_cat,yearsatcompany,yearsincurrentrole,yearssincelastpromotion,yearswithcurrmanager,sameasmonthlyincome,datebirth,salary,roledepartament,numberchildren,remotework
0,51,No,NaN,"684,0$",NaN,6,3,NaN,1,"162,0",1.0,Nothing,0,male,51.0,3,5,research director,3,NaN,19537.0,6462.0,7,Y,No,13,"3,0",3,Medium,NaN,0,NaN,5,3.0,Medium,20,NaN,15,15,"19537,0",1972,1000000000$,NaN,NaN,Yes
1,52,No,NaN,"699,0$",NaN,1,4,life sciences,1,"259,0",3.0,Medium,0,male,65.0,2,5,manager,3,NaN,19999.0,5678.0,0,NaN,NaN,14,"3,0",1,Nothing,NaN,1,34.0,5,3.0,Medium,33,NaN,11,9,"19999,0",1971,1000000000$,NaN,NaN,Yes


In [ ]:
df_data_copy.insert(7, 'education_cat', df_data_copy["education"].apply(categorizar_education))

In [ ]:
#"jobinvolvement"

def categorizar_jobinvolvement(valor):
    if valor == 4:
        return "very involved"
    elif valor == 3:
        return "involved"
    elif valor == 2:
        return "little involved"
    elif valor == 1:
        return "not involved"
    else:
        return "Unknown"

df_data_copy.head(2)

,age,attrition,businesstravel,dailyrate,department,distancefromhome,education,education_cat,educationfield,employeecount,employeenumber,environmentsatisfaction,environmentsatisfaction_cat,gender,gender_cat,hourlyrate,jobinvolvement,joblevel,jobrole,jobsatisfaction,maritalstatus,monthlyincome,monthlyrate,numcompaniesworked,over18,overtime,percentsalaryhike,performancerating,relationshipsatisfaction,relationshipsatisfaction_cat,standardhours,stockoptionlevel,totalworkingyears,trainingtimeslastyear,worklifebalance,worklifebalance_cat,yearsatcompany,yearsincurrentrole,yearssincelastpromotion,yearswithcurrmanager,sameasmonthlyincome,datebirth,salary,roledepartament,numberchildren,remotework,jobinvolvement_cat
0,51,No,NaN,"684,0$",NaN,6,3,college,NaN,1,"162,0",1.0,Nothing,0,male,51.0,3,5,research director,3,NaN,19537.0,6462.0,7,Y,No,13,"3,0",3,Medium,NaN,0,NaN,5,3.0,Medium,20,NaN,15,15,"19537,0",1972,1000000000$,NaN,NaN,Yes,involved
1,52,No,NaN,"699,0$",NaN,1,4,Unknown,life sciences,1,"259,0",3.0,Medium,0,male,65.0,2,5,manager,3,NaN,19999.0,5678.0,0,NaN,NaN,14,"3,0",1,Nothing,NaN,1,34.0,5,3.0,Medium,33,NaN,11,9,"19999,0",1971,1000000000$,NaN,NaN,Yes,little involved


In [ ]:
#df_data_copy.insert(17, 'jobinvolvement_cat', df_data_copy["jobinvolvement"].apply(categorizar_jobinvolvement))

ValueError: cannot insert jobinvolvement_cat, already exists

In [ ]:
#relationshipsatisfaction

In [ ]:
#df_data_copy[["education","educationfield","age","joblevel","jobrole", "hourlyrate","monthlyrate","yearsatcompany"]].sort_values(by = "hourlyrate", ascending=False).head(50)

In [ ]:
#df_data_copy[["jobinvolvement","environmentsatisfaction","jobsatisfaction"]].sort_values(by = "jobinvolvement", ascending=False).tail(50)

In [ ]:
#df_data_copy[["joblevel","jobrole", "hourlyrate","monthlyrate","yearsatcompany","education","educationfield","age"]].sort_values(by = "joblevel", ascending=False).head(50)

In [ ]:
#df_data_copy[["jobinvolvement","environmentsatisfaction","jobsatisfaction"]].sort_values(by = "jobsatisfaction", ascending=False).tail(50)

In [ ]:
#df_data_copy[["jobinvolvement","environmentsatisfaction","jobsatisfaction", "relationshipsatisfaction"]].sort_values(by = "relationshipsatisfaction", ascending=False).head(50)

In [ ]:
#df_data_copy[["stockoptionlevel", "yearsatcompany","totalworkingyears","joblevel","jobrole"]].sort_values(by = "stockoptionlevel", ascending=False).head(50)

In [ ]:
#df_data_copy[["worklifebalance","jobinvolvement","environmentsatisfaction","jobsatisfaction", "relationshipsatisfaction"]].sort_values(by = "worklifebalance", ascending=False).head(50)

#### 2.7 Duplicados (filas)

In [ ]:
df_data_copy.columns

Index(['age', 'attrition', 'businesstravel', 'dailyrate', 'department',
       'distancefromhome', 'education', 'education_cat', 'educationfield',
       'employeecount', 'employeenumber', 'environmentsatisfaction',
       'environmentsatisfaction_cat', 'gender', 'gender_cat', 'hourlyrate',
       'jobinvolvement', 'joblevel', 'jobrole', 'jobsatisfaction',
       'maritalstatus', 'monthlyincome', 'monthlyrate', 'numcompaniesworked',
       'over18', 'overtime', 'percentsalaryhike', 'performancerating',
       'relationshipsatisfaction', 'relationshipsatisfaction_cat',
       'standardhours', 'stockoptionlevel', 'totalworkingyears',
       'trainingtimeslastyear', 'worklifebalance', 'worklifebalance_cat',
       'yearsatcompany', 'yearsincurrentrole', 'yearssincelastpromotion',
       'yearswithcurrmanager', 'sameasmonthlyincome', 'datebirth', 'salary',
       'roledepartament', 'numberchildren', 'remotework',
       'jobinvolvement_cat'],
      dtype='object')

In [24]:
#Nro. de duplicados de todas las finals
df_data_copy.duplicated().sum()

43

In [25]:
df_data_copy.duplicated(keep=False).sum()

86

In [26]:
#Nro. de duplicados en base al employeenumber
df_data_copy["employeenumber"].duplicated().sum()

534

In [27]:
#Visualización de todos los duplicados 
df_data_copy[df_data_copy.duplicated(keep=False)].sort_values(by = ["jobrole","age","dailyrate"], ascending=False).head(50).T

,349,1592,884,1525,319,1562,324,1567,463,1505,82,1474,366,1609,369,1612,1002,1489,894,1535,338,1581,457,1499,301,1544,83,1475,469,1511,899,1540,989,1485,889,1530,875,1516,359,1602,988,1484,87,1479,341,1584,321,1564,332,1575
age,45,45,21,21,56,56,48,48,43,43,38,38,36,36,36,36,35,35,34,34,32,32,31,31,29,29,29,29,28,28,45,45,38,38,36,36,28,28,26,26,26,26,38,38,32,32,46,46,37,37
attrition,No,No,No,No,No,No,No,No,No,No,No,No,Yes,Yes,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,Yes,Yes,Yes,Yes,Yes,Yes,No,No,No,No,No,No,No,No
businesstravel,travel_rarely,travel_rarely,NaN,NaN,NaN,NaN,travel_rarely,travel_rarely,travel_frequently,travel_frequently,NaN,NaN,travel_rarely,travel_rarely,non-travel,non-travel,NaN,NaN,NaN,NaN,NaN,NaN,travel_rarely,travel_rarely,travel_frequently,travel_frequently,travel_frequently,travel_frequently,travel_frequently,travel_frequently,NaN,NaN,travel_frequently,travel_frequently,travel_rarely,travel_rarely,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,travel_rarely,travel_rarely,NaN,NaN,travel_rarely,travel_rarely
dailyrate,"954,0$","954,0$","895,0$","895,0$","906,0$","906,0$","163,0$","163,0$","1422,0$","1422,0$","119,0$","119,0$","530,0$","530,0$","1229,0$","1229,0$","1182,0$","1182,0$","1381,0$","1381,0$","929,0$","929,0$","196,0$","196,0$","442,0$","442,0$","1413,0$","1413,0$",nan$,nan$,"192,0$","192,0$","1186,0$","1186,0$","329,0$","329,0$","1157,0$","1157,0$",nan$,nan$,"342,0$","342,0$",nan$,nan$,"1018,0$","1018,0$","1485,0$","1485,0$","482,0$","482,0$"
department,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
distancefromhome,2,2,9,9,6,6,2,2,2,2,3,3,3,3,8,8,11,11,4,4,10,10,29,29,2,2,1,1,1,1,10,10,3,3,2,2,2,2,5,5,2,2,15,15,3,3,18,18,3,3
education,2,2,2,2,3,3,5,5,4,4,3,3,1,1,4,4,2,2,4,4,3,3,4,4,2,2,1,1,2,2,2,2,4,4,3,3,4,4,2,2,3,3,2,2,2,2,3,3,3,3
educationfield,NaN,NaN,NaN,NaN,life sciences,life sciences,NaN,NaN,life sciences,life sciences,life sciences,life sciences,life sciences,life sciences,technical degree,technical degree,marketing,marketing,NaN,NaN,marketing,marketing,marketing,marketing,NaN,NaN,NaN,NaN,life sciences,life sciences,life sciences,life sciences,NaN,NaN,NaN,NaN,medical,medical,medical,medical,life sciences,life sciences,life sciences,life sciences,life sciences,life sciences,medical,medical,other,other
employeecount,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1
employeenumber,"783,0","783,0",NaN,NaN,NaN,NaN,"595,0","595,0","1849,0","1849,0","307,0","307,0","967,0","967,0","990,0","990,0",NaN,NaN,"523,0","523,0","722,0","722,0",NaN,NaN,"388,0","388,0",NaN,NaN,"1927,0","1927,0","544,0","544,0",NaN,NaN,"501,0","501,0","440,0","440,0",NaN,NaN,"1053,0","1053,0",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [28]:
print("Cantidad de filas antes de eliminar duplicados:", len(df_data_copy))
df_data_copy = df_data_copy.drop_duplicates()
print("Cantidad de filas después de eliminar duplicados:", len(df_data_copy))


Cantidad de filas antes de eliminar duplicados: 1614
Cantidad de filas después de eliminar duplicados: 1571


In [29]:
df_data_copy.duplicated().sum()

0

In [30]:
df_data_copy[df_data_copy.duplicated(subset = 'employeenumber', keep=False)].sort_values(by = 'employeenumber', ascending=False).head(50).T

,1610,367,1608,365,1606,363,1605,362,361,1604,360,1603,357,1600,355,1598,1595,352,350,1593,348,1591,347,1590,346,1589,1585,342,340,1583,1582,339,1579,336,1578,335,331,1574,330,1573,329,1572,1571,328,1566,323,1482,90,1563,320
age,45,45,32,32,26,26,31,31,41,41,44,44,55,55,36,36,38,38,36,36,56,56,30,30,36,36,47,47,27,27,38,38,34,34,29,29,53,53,43,43,31,31,34,34,35,35,50,50,41,41
attrition,No,No,No,No,No,No,No,No,Yes,Yes,No,No,Yes,Yes,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No
businesstravel,non-travel,non-travel,NaN,NaN,travel_frequently,travel_frequently,travel_rarely,travel_rarely,NaN,NaN,NaN,NaN,NaN,NaN,travel_rarely,travel_rarely,NaN,NaN,non-travel,non-travel,travel_rarely,travel_rarely,NaN,NaN,NaN,NaN,NaN,NaN,travel_rarely,travel_rarely,travel_rarely,travel_rarely,NaN,NaN,NaN,NaN,NaN,NaN,travel_frequently,travel_frequently,travel_rarely,travel_rarely,NaN,NaN,travel_rarely,travel_rarely,travel_rarely,travel_rarely,travel_rarely,travel_rarely
dailyrate,"805,0$","805,0$","498,0$","498,0$","1283,0$","1283,0$","154,0$","154,0$","1085,0$","1085,0$","661,0$","661,0$","436,0$","436,0$","676,0$","676,0$","240,0$","240,0$","1434,0$","1434,0$",nan$,nan$,"501,0$","501,0$","427,0$","427,0$","703,0$","703,0$",nan$,nan$,"827,0$","827,0$","1442,0$","1442,0$","806,0$","806,0$","238,0$","238,0$",nan$,nan$,"525,0$","525,0$","258,0$","258,0$","1276,0$","1276,0$","1099,0$","1099,0$","645,0$","645,0$"
department,NaN,NaN,NaN,NaN,Sales,Sales,NaN,NaN,Research & Development,Research & Development,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Research & Development,Research & Development,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Research & Development,Research & Development,Sales,Sales,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Research & Development,Research & Development,Sales,Sales
distancefromhome,4,47,3,3,1,28,7,7,2,2,9,9,2,2,1,1,2,2,8,8,9,9,27,35,8,8,14,35,1,1,1,1,9,9,1,1,1,1,9,9,6,29,21,21,16,16,29,27,1,1
education,2,2,4,4,3,3,4,4,4,4,2,2,1,1,3,3,4,4,4,4,3,3,5,5,3,3,4,4,2,2,4,4,3,3,4,4,1,1,5,5,4,4,4,4,3,3,4,4,3,3
educationfield,NaN,NaN,medical,medical,medical,medical,NaN,NaN,life sciences,life sciences,NaN,NaN,medical,medical,other,other,life sciences,life sciences,NaN,NaN,NaN,NaN,marketing,marketing,NaN,NaN,marketing,marketing,life sciences,life sciences,life sciences,life sciences,medical,medical,life sciences,life sciences,NaN,NaN,NaN,NaN,medical,medical,NaN,NaN,life sciences,life sciences,life sciences,life sciences,marketing,marketing
employeecount,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1
employeenumber,"972,0","972,0","966,0","966,0","956,0","956,0","941,0","941,0","927,0","927,0","913,0","913,0","842,0","842,0","823,0","823,0","803,0","803,0","789,0","789,0","762,0","762,0","747,0","747,0","742,0","742,0","728,0","728,0","725,0","725,0","724,0","724,0","717,0","717,0","710,0","710,0","682,0","682,0","663,0","663,0","653,0","653,0","621,0","621,0","586,0","586,0","569,0","569,0","534,0","534,0"


In [ ]:
print("Cantidad de duplicados después de eliminar:", df_data_copy["employeenumber"].duplicated().sum())


Cantidad de duplicados después de eliminar: 0


In [ ]:
df_data_copy.duplicated().sum()

0

In [ ]:
df_data_copy.duplicated(keep=False).sum()

0

#### 2.8 Nulos

In [ ]:
#1.CATEGÓRICAS - Ana
# businesstravel
# department - muchos nulos
# education field
# marital status - 119
# overtime
# performancerating - 195 nulos
# Role in department - muchos nulos

In [ ]:
#La lista de columnas categoricas con nulos.
nulos_esta_cat = df_data_copy[df_data_copy.columns[df_data_copy.isnull().any()]].select_dtypes(include = "O").columns
print("Las columnas categóricas que tienen nulos son : \n ")
print(nulos_esta_cat)

In [ ]:
# El 'value_counts()' de cada una de las columnas categóricas que tienen nulos para saber como es la distribución de sus categorías
for col in nulos_esta_cat:
    print(f"La distribución de las categorías para la columna {col.upper()}")
    display(df_data_copy[col].value_counts() / df_data_copy.shape[0])
    print("........................")

Performance rating es la única que tendría posibilidades de cambiar a la moda, las otras tienen valores muy bajos y seria convenientes cmabiar por desconocido. 

In [ ]:
columnas_moda = ['performancerating']
columnas_desconocido = ['businesstravel', 'department', 'educationfield', 'employeenumber',
       'maritalstatus', 'over18', 'overtime', 'performancerating',
       'standardhours', 'yearsincurrentrole', 'sameasmonthlyincome',
       'roledepartament']

In [ ]:
#Columna moda
for columna in columnas_moda:
    moda = df_data_copy[col].mode()[0]    
    df_data_copy[columna] = df_data_copy[columna].fillna(moda)

print("Después del reemplazo usando 'fillna' quedan los siguientes nulos")
df_data_copy[columnas_moda].isnull().sum()

In [ ]:
#columna desconocido
for columna in columnas_desconocido:
    df_data_copy[columna] = df_data_copy[columna].fillna("desconocido") 

print("Después del reemplazo usando 'fillna' quedan los siguientes nulos")
df_data_copy[columnas_desconocido].isnull().sum()

In [ ]:
#2.NUMÉRICAS - Elena
# hourly_rate = 84 nulos
# Environmental satisfaction 
# monthlyincome
# worklifebalance - 108
# totalworkingyears
# numberofchildren

In [ ]:
#Columnas numéricas con nulos 
nulos_num = df_data_copy[df_data_copy.columns[df_data_copy.isnull().any()]].select_dtypes(include = np.number).columns
print(nulos_num)

In [ ]:
#% de nulos en las columnas numéricas con nulos
nl = (df_data_copy[nulos_num].isnull().sum() / df_data_copy.shape[0]) * 100
#nl.sort_values(ascending=False).reset_index()
nl.reset_index()

In [ ]:
#Valores atípicos
fig, axes = plt.subplots(nrows = 3, ncols = 2, figsize = (8,10)) 
axes = axes.flat

for indice, col in enumerate(nulos_num):
    sns.boxplot(x = col, data = df_data_copy, ax = axes[indice])
    plt.xlabel(col)  

plt.tight_layout()
fig.delaxes(axes[-1])
plt.show();

In [ ]:
#Para crear las columnas de iterativo y de knn automáticamente

#---------------ITERATIVE-----------------------------------
# palabras = ['environmentsatisfaction', 'hourlyrate', 'monthlyincome',
#        'totalworkingyears', 'worklifebalance', 'numberchildren']

# palabras_iterativas = [palabra + '_iterativo' for palabra in palabras]

# print(palabras_iterativas)

# #---------------KNN-----------------------------------

# palabras2 = ['environmentsatisfaction', 'hourlyrate', 'monthlyincome',
#        'totalworkingyears', 'worklifebalance', 'numberchildren']

# palabras_iterativas = [palabra + '_knn' for palabra in palabras2]

# print(palabras_iterativas)

# #---------------TODAS JUNTAS-----------------------------------

# pal = ['environmentsatisfaction', 'hourlyrate', 'monthlyincome',
#        'totalworkingyears', 'worklifebalance', 'numberchildren']

# pal2 = ['environmentsatisfaction_iterativo', 'hourlyrate_iterativo', 'monthlyincome_iterativo', 'totalworkingyears_iterativo', 'worklifebalance_iterativo', 'numberchildren_iterativo']


# pal3 = ['environmentsatisfaction_knn', 'hourlyrate_knn', 'monthlyincome_knn', 'totalworkingyears_knn', 'worklifebalance_knn', 'numberchildren_knn']


# lista_concatenada = [item for sublist in zip(pal, pal2, pal3) for item in sublist]

# print(lista_concatenada)

#### Decisiones: Nulos (numéricas)
- Number of children: borrarla
- Todas las demás: +5% nulos a 52% utilizar el iterate y knn imputer - atípicos en monthlyincome y totalworkingyears

In [ ]:
#ITERATIVE IMPUTER

imputer_iterative = IterativeImputer(max_iter =20, random_state=42)
imputer_iterative_imputado = imputer_iterative.fit_transform(df_data_copy[['environmentsatisfaction', 'hourlyrate', 'monthlyincome','totalworkingyears', 'worklifebalance']])
imputer_iterative_imputado

In [ ]:
df_data_copy[['environmentsatisfaction_iterativo', 'hourlyrate_iterativo', 'monthlyincome_iterativo', 'totalworkingyears_iterativo', 'worklifebalance_iterativo']] = imputer_iterative_imputado

In [ ]:
#KNN IMPUTER
imputer_knn = KNNImputer(n_neighbors=5)
imputer_knn_imputado = imputer_knn.fit_transform(df_data_copy[['environmentsatisfaction', 'hourlyrate', 'monthlyincome',
       'totalworkingyears', 'worklifebalance']])
imputer_knn_imputado

In [ ]:
df_data_copy[['environmentsatisfaction_knn', 'hourlyrate_knn', 'monthlyincome_knn', 'totalworkingyears_knn', 'worklifebalance_knn']] = imputer_knn_imputado

In [ ]:
#Comparamos las estadísticas generadas de por el iterative y el knn para decidir cuál coger (aquel cuya media y mediana sea más parecida a la original- En la columna con valores atípicos cogemos aquel cuya mediana sea más parecida a la original)
df_data_copy.describe()[['environmentsatisfaction', 'environmentsatisfaction_iterativo', 'environmentsatisfaction_knn', 'hourlyrate', 'hourlyrate_iterativo', 'hourlyrate_knn', 'monthlyincome', 'monthlyincome_iterativo', 'monthlyincome_knn', 'totalworkingyears', 'totalworkingyears_iterativo', 'totalworkingyears_knn', 'worklifebalance', 'worklifebalance_iterativo', 'worklifebalance_knn']].T

### Cuál coger?
- env.satif_iterativo
- hourlyrate_iterativo
- Monthlyincome?? knn es quizás mejor!mejor representativo la moda q la media pq tiene muchos valores 
- totalworkingyears_iterativo
- worklifebalance_iterativo

In [ ]:
##Pasos q faltan: 1.eliminar las columnas que no queremos y 2.cambiar el nombre de las columnas de knn e iterativo elegidas quitándoles las palabras "knn/iterativo"

## Nuestras decisiones
Distance from home - cambiar los valores negativos a positivos -- Nuestra decisión es que hubo un error de inserción pero q son valores positivos

## Martes 5.12
- Añadir funcions de todas y runearlas
- Ver duplicados y borrarlos
- Discutir nulos y decidir acción
- Borrar columnas

notas:
- El employee number no nos sirve - muchos nulos y muchos duplicados (534)
- 